In [ ]:
from pulp import *
import pulp
import pandas as pd
import numpy as np

In [ ]:
# Read the Excel file into a DataFrame
df = pd.read_excel('file.xlsx')

# Convert the DataFrame to a NumPy array
data = df.values
products = []

for row in data:
    predictedProfits = row[0]
    predictedProductQuantities = row[1]
    dimension = row[2]
    productName = row[3]

    product = {'predictedProfits': predictedProfits, 'predictedProductQuantities': predictedProductQuantities, 'dimension': dimension, 'productName': productName}
    products.append(product)

print(products)

In [ ]:
def calculate_profit(products):
    for product in products:
        profit = product['predictedProfits'] / product['predictedProductQuantities']
        product['productProfit'] = profit
    return products

In [ ]:
productTest = calculate_profit(products)
print(productTest) 

In [ ]:
# Creation of the ILP model 
model = pulp.LpProblem("ILP_Problem", pulp.LpMaximize)

In [ ]:
# Define the decision variables
decisionVariables = []
def createDictionaryWithDecisionVariable():
    for product in products:
        decisionVariable = {product['productName']:  pulp.LpVariable(product['productName'], lowBound=0, cat='Integer')}
        decisionVariables.append(decisionVariable)
        
createDictionaryWithDecisionVariable()
print(decisionVariables)

In [ ]:
# function to calculate objective function

def objectiveFunction(products):
    model = 0
    count = 0
    for product in products:
        model = model + (product['predictedProfits'] * decisionVariables[count][product['productName']])
        count = count + 1
    return model

#objective function
model += objectiveFunction(products)

In [ ]:
#add user input for the multiple variables

maxTotalAmount = int(input("Please insert maximum  total amount of products: "))
minTotalAmount = int(input("Please insert minimum total amount of products: "))

#create two list with min and max amount per product by user insert

minAmountProduct = int(input("Please insert minimum  amount of products : "))
maxAmountProduct  = int(input("Please insert maximum  amount of products : "))


In [ ]:
# Define the constraints

#restriction to total amount of products

def maxMinTotalProductsRestriction(products):
    restrictFunction = 0
    count = 0
    for product in products:
        restrictFunction = restrictFunction + decisionVariables[count][product['productName']]
        count = count + 1
    return restrictFunction
    
model += maxMinTotalProductsRestriction(products) <= maxTotalAmount #restricao de quantidade maximo total de produtos
model += maxMinTotalProductsRestriction(products) >= minTotalAmount #restricao de valor minimo total de produtos

# variety of products

#restriction to maximum quantity of products
def maxMinQuantityProductsRestriction(products,model):
    count = 0
    for product in products:
        model += decisionVariables[count][product['productName']] <= maxAmountProduct
        model += decisionVariables[count][product['productName']] >= minAmountProduct
        count = count + 1
        
maximumProductsRestriction(products, model)


#restriction to dimensions of products

#dimensao da loja
storeDimension = 2000

def maxDimensionProductsRestriction(products):
    count = 0
    restrictFunction = 0
    for product in products:
        restrictFunction = restrictFunction + (product['dimension'] * decisionVariables[count][product['productName'])

                                                                                               
model += maxDimensionProductsRestriction(products) <= storeDimension #restricao de dimensões de produtos, menor que dimensao todal da loja


In [ ]:
# Solve the problem
status = model.solve()

In [ ]:
print(status)

In [ ]:
# Print the solution
#print("Solution:")
#print("x =", x.value())
#print("y =", y.value())